# Fast Discrete Fourier Transform

In [1]:
import numpy as np

In [2]:
def fft(seq, inverse=False):
  n = len(seq)

  if n == 1:
    return seq

  w = np.exp(np.pi * 2j / n)

  if not inverse:
    w **= -1

  even = seq[0:][::2]
  odd = seq[1:][::2]

  a = fft(even, inverse)
  b = fft(odd, inverse)
  y = np.empty(n, dtype='complex_')

  for i in range(n // 2):
    x = w**i
    y[i] = a[i] + b[i] * x
    y[i + (n // 2)] = a[i] - b[i] * x

  if inverse:
    y /= 2

  return y

In [5]:
def next_power_of_2(x):
  return 1 if x == 0 else 2**(x - 1).bit_length()

def convolution(seq1, seq2):
  final_length = len(seq1) + len(seq2) - 1
  n = next_power_of_2(final_length)

  seq1 = np.pad(seq1, (0, n - len(seq1)), 'constant', constant_values=0)
  seq2 = np.pad(seq2, (0, n - len(seq2)), 'constant', constant_values=0)

  seq1_dft = fft(seq1)
  seq2_dft = fft(seq2)

  product = np.multiply(seq1_dft, seq2_dft)

  return fft(product, inverse=True)[:final_length]

## Randomized Testing

In [6]:
def test(seq1, seq2):
  result = convolution(seq1, seq2)
  correct = np.convolve(seq1, seq2)

  assert(len(correct) == len(result))
  np.testing.assert_allclose(result, correct)

for _ in range(1000):
  size1 = np.random.default_rng().integers(low=4, high=50)
  size2 = np.random.default_rng().integers(low=4, high=50)
  seq1 = np.random.default_rng().uniform(low=-10000, high=10000, size=size1)
  seq2 = np.random.default_rng().uniform(low=-10000, high=10000, size=size2)
  test(seq1, seq2)